<a href="https://colab.research.google.com/github/felixlys/Credit-Analysis_Stepwise-Logistic-Regression/blob/main/LogisticRegression_For_Credit_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression for Credit Analysis

In [1]:
library(tidyverse)
library(nortest)
library(psych)
library(qqplotr)
library(lm.beta)
library(lmtest)
library(readxl)
library(car)
library(semPlot)
library(interactions)
library(tidymodels)
library(MASS)
library(pscl)
library(generalhoslem)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: Error in library(nortest): there is no package called ‘nortest’


Diberikan data mengenai data pribadi dari 45000 debitur yang pernah mengajukan kredit beserta histori status gagal kredit (1) atau  berhasil kredit (0). Berikut merupakan variabel-variabel nya :\\

**UMUR** : umur debitur\
**JK** : Jenis Kelamin\
**PENDIDIKAN**: Pendidikan Terakhir\
**GAJI**: Pendapatan debitur per tahun(dalam dollar)\
**LAMA BEKERJA**: Lama debitur bekerja dalam tahun\
**RUMAH**: status kepemilikan rumah debitur\
**PINJAMAN**: Jumlah pinjaman\
**PROFESI**: Profesi debitur\
**BUNGA**: Interest Rate pinjaman\
**PERSENTASE**: Debt to Income\
**HISTORI**: Berapa kali debitur pernah melakukan pinjaman\
**SCORE**: Skor Kredit\
**LUNAS**: Berhasil atau tidaknya kredit (0: Berhasil Kredit, 1: Tidak Berhasil Kredit).





In [28]:

data<-read.csv("data_Historis_Hutang.csv")
head(data)
str(data)

,Umur,JK,PENDIDIKAN,GAJI,LAMA.KERJA,RUMAH,PINJAMAN,PROFESI,BUNGA,PERSENTASE,HISTORI,SCORE,LUNAS
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<dbl>,<int>,<int>,<int>
1,22,female,Master,71948,0,RENT,35000,PERSONAL,16.02,0.49,3,561,1
2,21,female,High School,12282,0,OWN,1000,EDUCATION,11.14,0.08,2,504,0
3,25,female,High School,12438,3,MORTGAGE,5500,MEDICAL,12.87,0.44,3,635,1
4,23,female,Bachelor,79753,0,RENT,35000,MEDICAL,15.23,0.44,2,675,1
5,24,male,Master,66135,1,RENT,35000,MEDICAL,14.27,0.53,4,586,1
6,21,female,High School,12951,0,OWN,2500,VENTURE,7.14,0.19,2,532,1


'data.frame':	45000 obs. of  13 variables:
 $ Umur      : int  22 21 25 23 24 21 26 24 24 21 ...
 $ JK        : chr  "female" "female" "female" "female" ...
 $ PENDIDIKAN: chr  "Master" "High School" "High School" "Bachelor" ...
 $ GAJI      : int  71948 12282 12438 79753 66135 12951 93471 95550 100684 12739 ...
 $ LAMA.KERJA: int  0 0 3 0 1 0 1 5 3 0 ...
 $ RUMAH     : chr  "RENT" "OWN" "MORTGAGE" "RENT" ...
 $ PINJAMAN  : int  35000 1000 5500 35000 35000 2500 35000 35000 35000 1600 ...
 $ PROFESI   : chr  "PERSONAL" "EDUCATION" "MEDICAL" "MEDICAL" ...
 $ BUNGA     : num  16 11.1 12.9 15.2 14.3 ...
 $ PERSENTASE: num  0.49 0.08 0.44 0.44 0.53 0.19 0.37 0.37 0.35 0.13 ...
 $ HISTORI   : int  3 2 3 2 4 2 3 4 2 3 ...
 $ SCORE     : int  561 504 635 675 586 532 701 585 544 640 ...
 $ LUNAS     : int  1 0 1 1 1 1 1 1 1 1 ...


Pada tahap selanjutnya, variabel JK,PENDIDIKAN,RUMAH,PROFESI,LUNAS perlu untuk dijadikan variabel faktor


In [29]:
data<-data%>%
  mutate(JK=as.factor(JK),
PENDIDIKAN=as.factor(PENDIDIKAN),
RUMAH=as.factor(RUMAH),
PROFESI=as.factor(PROFESI),
LUNAS=as.factor(LUNAS))%>%
  na.omit()
str(data)

'data.frame':	45000 obs. of  13 variables:
 $ Umur      : int  22 21 25 23 24 21 26 24 24 21 ...
 $ JK        : Factor w/ 2 levels "female","male": 1 1 1 1 2 1 1 1 1 1 ...
 $ PENDIDIKAN: Factor w/ 5 levels "Associate","Bachelor",..: 5 4 4 2 5 4 2 4 1 4 ...
 $ GAJI      : int  71948 12282 12438 79753 66135 12951 93471 95550 100684 12739 ...
 $ LAMA.KERJA: int  0 0 3 0 1 0 1 5 3 0 ...
 $ RUMAH     : Factor w/ 4 levels "MORTGAGE","OTHER",..: 4 3 1 4 4 3 4 4 4 3 ...
 $ PINJAMAN  : int  35000 1000 5500 35000 35000 2500 35000 35000 35000 1600 ...
 $ PROFESI   : Factor w/ 6 levels "DEBTCONSOLIDATION",..: 5 2 4 4 4 6 2 4 5 6 ...
 $ BUNGA     : num  16 11.1 12.9 15.2 14.3 ...
 $ PERSENTASE: num  0.49 0.08 0.44 0.44 0.53 0.19 0.37 0.37 0.35 0.13 ...
 $ HISTORI   : int  3 2 3 2 4 2 3 4 2 3 ...
 $ SCORE     : int  561 504 635 675 586 532 701 585 544 640 ...
 $ LUNAS     : Factor w/ 2 levels "0","1": 2 1 2 2 2 2 2 2 2 2 ...


Selanjutnya dibentuk model Regresi Logistik dengan  Variabel LUNAS sebagai variabel dependen dan variabel independen nya adalah Umur,JK,Pendidikan,GAJI,LAMA KERJA,RUMAH,PINJAMAN,PROFESI,BUNGA,PERSENTASE,HISTORI,SCORE

In [30]:
reglog<-glm(LUNAS~Umur+JK+PENDIDIKAN+GAJI+LAMA.KERJA+RUMAH+PINJAMAN+PROFESI+BUNGA+PERSENTASE+HISTORI+SCORE,data=data,family=binomial(link='logit'))
summary(reglog)


Call:
glm(formula = LUNAS ~ Umur + JK + PENDIDIKAN + GAJI + LAMA.KERJA + 
    RUMAH + PINJAMAN + PROFESI + BUNGA + PERSENTASE + HISTORI + 
    SCORE, family = binomial(link = "logit"), data = data)

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)            -6.708e+00  2.738e-01 -24.497  < 2e-16 ***
Umur                    6.568e-03  8.762e-03   0.750 0.453491    
JKmale                  8.960e-03  2.820e-02   0.318 0.750709    
PENDIDIKANBachelor      1.816e-02  3.755e-02   0.484 0.628623    
PENDIDIKANDoctorate     7.465e-02  1.240e-01   0.602 0.547269    
PENDIDIKANHigh School   2.598e-02  3.856e-02   0.674 0.500453    
PENDIDIKANMaster        1.768e-02  4.532e-02   0.390 0.696388    
GAJI                    7.493e-07  1.964e-07   3.816 0.000136 ***
LAMA.KERJA             -8.966e-03  7.764e-03  -1.155 0.248189    
RUMAHOTHER              7.675e-01  2.456e-01   3.125 0.001779 ** 
RUMAHOWN               -1.430e+00  8.720e-02 -16.402  < 2e-1

Untuk menguji kelayakan model regresi logisik, seperti hal nya pada regresi linier yang perlu dilakukan uji kenormalan residual, maka model regresi logistik reglog harus diuji dengan Uji kelayakan Model Hosmer- Lemeshow.

In [31]:
install.packages("generalhoslem")
library(generalhoslem)
logitgof(data$LUNAS,fitted((reglog)))

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)




	Hosmer and Lemeshow test (binary model)

data:  data$LUNAS, fitted((reglog))
X-squared = 106.13, df = 8, p-value < 2.2e-16


Tampak bahwa pval < 0.05 sehingga model regresi logistik reglog belum dikatakan layak. Tahap berikutnya, perlu dilakukan eleminasi data outlier dengan menggunakan, menggunakan Residual Deviance (Deviance Residuals).


In [32]:
deviance_residuals <- residuals(reglog, type = "deviance")
threshold <- mean(deviance_residuals) + 3 * sd(deviance_residuals)
outliers <- which(abs(deviance_residuals) > threshold)
data_clean <- data[-outliers, ]
dim(data_clean)

[1] 44521    13

Selanjutnya, dapatkan model regresi logistik untuk data_clean,

In [33]:
reglog1<-glm(LUNAS~Umur+JK+PENDIDIKAN+GAJI+LAMA.KERJA+RUMAH+PINJAMAN+PROFESI+BUNGA+PERSENTASE+HISTORI+SCORE,data=data_clean,family=binomial(link='logit'))
summary(reglog1)


Call:
glm(formula = LUNAS ~ Umur + JK + PENDIDIKAN + GAJI + LAMA.KERJA + 
    RUMAH + PINJAMAN + PROFESI + BUNGA + PERSENTASE + HISTORI + 
    SCORE, family = binomial(link = "logit"), data = data_clean)

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)            -8.092e+00  2.944e-01 -27.490  < 2e-16 ***
Umur                    1.267e-02  9.336e-03   1.357 0.174711    
JKmale                 -2.553e-03  3.007e-02  -0.085 0.932333    
PENDIDIKANBachelor      9.528e-03  3.999e-02   0.238 0.811673    
PENDIDIKANDoctorate     9.148e-02  1.324e-01   0.691 0.489623    
PENDIDIKANHigh School   7.747e-03  4.113e-02   0.188 0.850588    
PENDIDIKANMaster       -5.438e-03  4.836e-02  -0.112 0.910467    
GAJI                    5.804e-07  2.179e-07   2.663 0.007744 ** 
LAMA.KERJA             -1.358e-02  8.278e-03  -1.640 0.100976    
RUMAHOTHER              9.218e-01  2.594e-01   3.554 0.000379 ***
RUMAHOWN               -1.695e+00  9.989e-02 -16.973  

In [34]:
#Uji kelayakan Model Hosmer- Lemeshow.
library(generalhoslem)
logitgof(data_clean$LUNAS,fitted((reglog1)))


	Hosmer and Lemeshow test (binary model)

data:  data_clean$LUNAS, fitted((reglog1))
X-squared = 301.91, df = 8, p-value < 2.2e-16


Masih belum layak. Oleh karena itu, cara lain yang bisa digunakan adalah meregresi data menggunakan stepwise regression, yakni menggunakan variabel variabel yang signifikan terhadap data dan membuang variabel yang tidak signifikan thd data.

In [35]:
stepreg1<-step(reglog1)

Start:  AIC=28804.43
LUNAS ~ Umur + JK + PENDIDIKAN + GAJI + LAMA.KERJA + RUMAH + 
    PINJAMAN + PROFESI + BUNGA + PERSENTASE + HISTORI + SCORE



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


             Df Deviance   AIC
- PENDIDIKAN  4    28761 28797
- JK          1    28760 28802
- SCORE       1    28761 28803
- HISTORI     1    28761 28803
- Umur        1    28762 28804
<none>             28760 28804
- LAMA.KERJA  1    28763 28805
- GAJI        1    28765 28807
- PINJAMAN    1    29276 29318
- PROFESI     5    29828 29862
- RUMAH       3    30884 30922
- PERSENTASE  1    31310 31352
- BUNGA       1    34516 34558

Step:  AIC=28797.01
LUNAS ~ Umur + JK + GAJI + LAMA.KERJA + RUMAH + PINJAMAN + PROFESI + 
    BUNGA + PERSENTASE + HISTORI + SCORE



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


             Df Deviance   AIC
- JK          1    28761 28795
- SCORE       1    28761 28795
- HISTORI     1    28762 28796
- Umur        1    28763 28797
<none>             28761 28797
- LAMA.KERJA  1    28764 28798
- GAJI        1    28765 28799
- PINJAMAN    1    29277 29311
- PROFESI     5    29829 29855
- RUMAH       3    30885 30915
- PERSENTASE  1    31312 31346
- BUNGA       1    34517 34551

Step:  AIC=28795.02
LUNAS ~ Umur + GAJI + LAMA.KERJA + RUMAH + PINJAMAN + PROFESI + 
    BUNGA + PERSENTASE + HISTORI + SCORE



...

             Df Deviance   AIC
- HISTORI     1    28762 28792
- Umur        1    28763 28793
<none>             28761 28793
- LAMA.KERJA  1    28764 28794
- GAJI        1    28766 28796
- PINJAMAN    1    29277 29307
- PROFESI     5    29830 29852
- RUMAH       3    30885 30911
- PERSENTASE  1    31312 31342
- BUNGA       1    34517 34547

Step:  AIC=28792.1
LUNAS ~ Umur + GAJI + LAMA.KERJA + RUMAH + PINJAMAN + PROFESI + 
    BUNGA + PERSENTASE



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


             Df Deviance   AIC
- Umur        1    28763 28791
<none>             28762 28792
- LAMA.KERJA  1    28765 28793
- GAJI        1    28767 28795
- PINJAMAN    1    29278 29306
- PROFESI     5    29832 29852
- RUMAH       3    30886 30910
- PERSENTASE  1    31312 31340
- BUNGA       1    34517 34545

Step:  AIC=28791.23
LUNAS ~ GAJI + LAMA.KERJA + RUMAH + PINJAMAN + PROFESI + BUNGA + 
    PERSENTASE



Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


             Df Deviance   AIC
<none>             28763 28791
- LAMA.KERJA  1    28768 28794
- GAJI        1    28768 28794
- PINJAMAN    1    29279 29305
- PROFESI     5    29836 29854
- RUMAH       3    30888 30910
- PERSENTASE  1    31313 31339
- BUNGA       1    34517 34543


In [36]:
summary(stepreg1)


Call:
glm(formula = LUNAS ~ GAJI + LAMA.KERJA + RUMAH + PINJAMAN + 
    PROFESI + BUNGA + PERSENTASE, family = binomial(link = "logit"), 
    data = data_clean)

Coefficients:
                         Estimate Std. Error z value Pr(>|z|)    
(Intercept)            -7.918e+00  9.755e-02 -81.164  < 2e-16 ***
GAJI                    6.237e-07  2.146e-07   2.906 0.003657 ** 
LAMA.KERJA             -5.638e-03  2.557e-03  -2.205 0.027440 *  
RUMAHOTHER              9.199e-01  2.592e-01   3.550 0.000386 ***
RUMAHOWN               -1.698e+00  9.987e-02 -16.998  < 2e-16 ***
RUMAHRENT               1.114e+00  3.594e-02  31.003  < 2e-16 ***
PINJAMAN               -1.116e-04  3.568e-06 -31.273  < 2e-16 ***
PROFESIEDUCATION       -1.103e+00  4.979e-02 -22.161  < 2e-16 ***
PROFESIHOMEIMPROVEMENT -1.708e-02  5.522e-02  -0.309 0.757050    
PROFESIMEDICAL         -3.490e-01  4.645e-02  -7.515  5.7e-14 ***
PROFESIPERSONAL        -8.224e-01  5.099e-02 -16.127  < 2e-16 ***
PROFESIVENTURE         -1.291e+

Tampak bahwa variabel yang berpengaruh secara signifikan adalah :  GAJI,LAMA.KERJA,RUMAH, PINJAMAN,PROFESI,BUNGA, PERSENTASE, dan yang tidak digunakan adalah Umur,JK,PENDIDIKAN,HISTORI,SCORE. Selanjutnya, dilakukan uji kelayakan model

In [37]:
#Uji kelayakan Model Hosmer- Lemeshow.
library(generalhoslem)
logitgof(data_clean$LUNAS,fitted((stepreg1)))


	Hosmer and Lemeshow test (binary model)

data:  data_clean$LUNAS, fitted((stepreg1))
X-squared = 304.34, df = 8, p-value < 2.2e-16


Model stepreg1 masih belum layak karena pvalue < 0.05. Namun, untuk asumsi multikolonieritas nya sudah terpenihi.

In [38]:
install.packages("car")
library(car)
vif(stepreg1)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



,GVIF,Df,GVIF^(1/(2*Df))
GAJI,1.229947,1,1.109030
LAMA.KERJA,1.047935,1,1.023687
RUMAH,1.142733,3,1.022486
PINJAMAN,2.237148,1,1.495710
PROFESI,1.073733,5,1.007140
BUNGA,1.156955,1,1.075618
PERSENTASE,2.306434,1,1.518695


Signifikasi model, kita harus menunjukkan bahwa nilai $G2>qchisq(0.95,7)$ karena ada 7 variabel.

In [39]:
install.packages("pscl")
library(pscl)
pR2(stepreg1)
qchisq(0.95,7)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



fitting null model for pseudo-r2


llh       llhNull            G2      McFadden          r2ML 
-1.438161e+04 -2.311894e+04  1.747464e+04  3.779293e-01  3.246360e-01 
         r2CU 
 5.025036e-01

[1] 14.06714

Tampak bahwa $G2=11169.2251497919>14.0671404493402$. Sehingga, model nya terbukti signifikan. Selanjutnya, dilakukan uji T

In [40]:
#Uji T/uji Watt, uji T pada regresi linier
summary(stepreg1)$coefficient%>%
  round(4)

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-7.9177,0.0976,-81.1645,0.0000
GAJI,0.0000,0.0000,2.9063,0.0037
LAMA.KERJA,-0.0056,0.0026,-2.2052,0.0274
RUMAHOTHER,0.9199,0.2592,3.5497,0.0004
RUMAHOWN,-1.6976,0.0999,-16.9984,0.0000
RUMAHRENT,1.1142,0.0359,31.0032,0.0000
PINJAMAN,-0.0001,0.0000,-31.2728,0.0000
PROFESIEDUCATION,-1.1034,0.0498,-22.1614,0.0000
PROFESIHOMEIMPROVEMENT,-0.0171,0.0552,-0.3094,0.7570
PROFESIMEDICAL,-0.3490,0.0464,-7.5147,0.0000


Perhatikan variabel/faktor yang memiliki nilai $Pr(>|z|)= 0$  memiliki berpengaruh besar terhadap keberhasilan kredit. Selanjutnya, dilakukan penghitungan ODD RATIO untuk menentukan tingkat resiko seseorang tidak berhasil kredit(nilai 1 pada variabel LUNAS).


In [41]:
stepreg1$coefficients%>%
  data.frame()%>%
  dplyr::rename(koefisien='.')%>%
  mutate(oddratio=round(exp(koefisien),2))

,koefisien,oddratio
,<dbl>,<dbl>
(Intercept),-7.917676e+00,0.00
GAJI,6.236838e-07,1.00
LAMA.KERJA,-5.637830e-03,0.99
RUMAHOTHER,9.199304e-01,2.51
RUMAHOWN,-1.697649e+00,0.18
RUMAHRENT,1.114241e+00,3.05
PINJAMAN,-1.115793e-04,1.00
PROFESIEDUCATION,-1.103387e+00,0.33
PROFESIHOMEIMPROVEMENT,-1.708227e-02,0.98


Berdasarkan analisis dari nilai UjiT dan Odd Ratio diperoleh informasi penting sebagai berikut ⁉
1.  Faktor RUMAHOWN: Apabila debitur memiliki rumah sendiri, maka kemungkinan untuk gagal kredit adalah 0.18 kali dibandingkan dengan debitur yang bukan memiliki rumah sendiri. Dengan kata lain, memiliki rumah sendiri mengurangi risiko gagal kredit sebesar 82%
2. Faktor RUMAHRENT: Apabila debitur tinggal di rumah sewa, maka kemungkinan untuk gagal kredit adalah 3.05 kali dibandingkan dengan debitur yang tidak tinggal di rumah sewa. Dengan kata lain,  jika debitur yang tinggal di rumah sewa maka resiko gagal kredit meningkat 3 kali lipat.
3. Variabel PINJAMAN : Setiap naiknya 1 dollar jumlah pinjaman yang disetujui maka meningkatkan resiko gagal kredit sebanyak 1 kali.
4. Faktor PROFESIEDUCATION : Apabila debitur berprofesi di bidang Edukasi maka kemungkinan untuk gagal kredit adalah 0.33 kali dibandingkan dengan debitur yang bukan berprofesi di bidang Edukasi.
5. Faktor PROFESIHOMEIMPROVEMENT :pabila debitur berprofesi di bidang HOME IMPROVEMENT maka kemungkinan untuk gagal kredit adalah 0.98 kali dibandingkan dengan debitur yang bukan berprofesi di bidang HOME IMPROVEMENT.
6. Variabel BUNGA : Setiap kenaikan 1 persen bunga kredit meningkatkan maka meningkatkan resiko gagal kredit sebanyak 1.5 kali lipat.


Setelah dilakukan pengkontruksian model dilakukan prediksi model regresi logistik berdasarkan data yang diberikan

In [42]:
data_clean$prediksi<-round(predict(stepreg1,data=data_clean,type='response'))
Result<-data_clean%>%
  dplyr::select(LUNAS,prediksi)%>%
  table()%>%
  data.frame()
Result

LUNAS,prediksi,Freq
<fct>,<fct>,<int>
0,0,33056
1,0,4433
0,1,1935
1,1,5097


Didapat akurasi model,
1. Menebak orang yang berhasil kredit,
$$ACC_{berhasil}=\frac{33056}{33056+1935}*100\%=94.470\%$$,
2. Menebak orang yang gagal kredit,
$$ACC_{gagal}=\frac{5097}{5097+4433}*100\%=53.483\%$$,
3. Keakuratan menebak benar,
$$ACC=\frac{33056+5097}{33056+1935+5097+4433}\times 100\%=85.696\%$$



In [51]:
Result1<-data_clean%>%
  dplyr::select(LUNAS,prediksi)

In [52]:
install.packages("writexl")
library(writexl)
write_xlsx(Result1, "DataPrednLunas.xlsx")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

